## Post Import in Group

This demonstrates the process of deploying a new Power BI Model as a PBIX file to a Power BI Workspace.  This leverages the "large file" method of intermediately loading the PBIX file to an Azure Storage Blob.  Fortunatley there is a handy method on the API to create a blob location with a sas token for us.

The general flow of this process is:
1. Authenticate the Service Principal.
1. Generate a temporary upload location
1. use the blob API to upload the file to teh temp location
1. Call the Imports API passing in the path to the temporary blob location.

The documentation for the relevant APIs can be found here:
- https://learn.microsoft.com/en-us/rest/api/power-bi/imports/create-temporary-upload-location-in-group
- https://learn.microsoft.com/en-us/rest/api/power-bi/imports/post-import-in-group

In [ ]:
pip install requests msal jsons azure-storage-blob requests_toolbelt

In [ ]:
#This leverages the code encapsulated in services/aadservice.py that encapsulates the service principle login
from services.aadservice import AadService
aad_token = AadService.get_access_token()

In [ ]:
workspace_id="16453fea-cb0f-4ef4-b1b2-bede9d3b92be"
display_name="NYCYellow.pbix"

In [ ]:
import requests
import json
from azure.storage.blob import BlobClient

temp_url = ""

#Part 1:  Request a temporary blob location from the API.
api_url = f'https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/imports/createTemporaryUploadLocation'
headers =  {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + aad_token
} 

api_response = requests.post(api_url, headers=headers)
#error handling for createTemporaryUplodadLocation
if api_response.status_code != 200:
    description = f'Error requesting temp location:\n  -Status Code:\t{api_response.status_code}\n  -Reason:\t{api_response.reason}\n  -RequestId:\t{api_response.headers.get("RequestId")}\n  -Text:\t{api_response.text}'
    print(description)
else:
    api_response = json.loads(api_response.text)
    temp_url = api_response['url']


In [ ]:
#Part 2:  Now that we have the temporary location, upload the file to it via the BlobClient
if temp_url != '':
    client = BlobClient.from_blob_url(temp_url)
    with open(file_location, 'rb') as data:
        client.upload_blob(data)
    print(f"{file_location} uploaded")
else:
    print("No sas url specified.")


In [ ]:
#Part 3:   Call the import API with the temp location as 'fileUrl' in the body of the API call
api_url = f"https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/imports?datasetDisplayName={display_name}&nameConflict=Ignore&skipReport=false&overrideReportLabel=true&overrideModelLabel=true"

body = {
    "fileUrl": temp_url
}

api_response = requests.post(
    url=api_url,
    headers=headers,
    data=json.dumps(body)
)

if api_response.status_code != 200:
    description = f'Error importing file:\n  -Status Code:\t{api_response.status_code}\n  -Reason:\t{api_response.reason}\n  -RequestId:\t{api_response.headers.get("RequestId")}\n  -Text:\t{api_response.text}'
    print(description)
else:
    api_response = json.loads(api_response.text)
    print(json.dumps(api_response,indent=2))
